In [ ]:
"""
把點雲進行三維重建
input: 點雲(.csv), 相機係數
這份是從 Simple, Accurate, and Robust Projector-Camera Calibration(https://ieeexplore.ieee.org/document/6375029)
這篇paper給的.cpp code 裡面改過來的
"""

In [1]:
import numpy as np
import cv2
import csv
import open3d as o3d

In [2]:
class Pointcloud:
    def __init__(self):
        self.points = np.empty((0))
        self.colors = np.empty((0))
        self.normals = np.empty((0))
    def clear(self):
        self.points = np.empty((0))
        self.colors = np.empty((0))
        self.normals = np.empty((0))

    def init_points(self, row, column):
        self.points = np.full((row, column, 3), np.nan)  # CV_32FC3
    def init_colors(self, row, column):
        self.colors = np.full((row, column, 3), 255)  # CV_8UC3
    def init_normals(self, row, column):
        self.normals = np.full((row, column, 3), np.nan)  # CV_32FC3


In [3]:
# Calibration parameter
class CalibrationData:
    def __init__(self, cam_k, cam_kc, proj_k, proj_kc, R, T,
                 cam_err, proj_err, stereo_err):
        self.cam_k = cam_k
        self.cam_kc = cam_kc
        self.proj_k = proj_k
        self.proj_kc = cam_kc
        self.R = R
        self.T = T
        self.cam_err = cam_err
        self.proj_err = proj_err
        self.stereo_err = stereo_err

In [4]:
# cv::Point3d scan3d::approximate_ray_intersection(const cv::Point3d & v1, const cv::Point3d & q1,
#                                                     const cv::Point3d & v2, const cv::Point3d & q2,
#                                                     double * distance, double * out_lambda1, double * out_lambda2)
def approximate_ray_intersection(v1, q1, v2, q2):
    v1tv1 = np.dot(v1, v1.transpose())  # double
    v2tv2 = np.dot(v2, v2.transpose())
    v1tv2 = np.dot(v1, v2.transpose())
    v2tv1 = np.dot(v2, v1.transpose())
    detV = v1tv1*v2tv2 - v1tv2*v2tv1 #double
    # Vinv.at<double>(0,0) = v2tv2/detV;  Vinv.at<double>(0,1) = v1tv2/detV;
    # Vinv.at<double>(1,0) = v2tv1/detV; Vinv.at<double>(1,1) = v1tv1/detV;
    Vinv = np.array([[v2tv2, v1tv2], [v2tv1, v1tv1]])
    Vinv = Vinv/detV

    q2_q1 = q2-q1
    # double Q1 = v1.x*q2_q1.x + v1.y*q2_q1.y + v1.z*q2_q1.z;
    # double Q2 = -(v2.x*q2_q1.x + v2.y*q2_q1.y + v2.z*q2_q1.z);
    Q1 = v1[0]*q2_q1[0]+v1[1]*q2_q1[1]+v1[2]*q2_q1[2]
    Q2 = -(v2[0]*q2_q1[0]+v2[1]*q2_q1[1]+v2[2]*q2_q1[2])
    lambda1 = (v2tv2*Q1+v1tv2*Q2)/detV#double
    lambda2 = (v2tv1*Q1+v1tv1*Q2)/detV
    p1 = lambda1*v1+q1
    p2 = lambda2*v2+q2
    p = 0.5*(p1+p2)
    # if distance is not None:  # ???
    distance = cv2.norm(p2-p1)
    # 中間有改lamda的東西應該沒關西嗎???
    return p, distance

In [5]:
# void scan3d::triangulate_stereo(const cv::Mat & K1, const cv::Mat & kc1, const cv::Mat & K2, const cv::Mat & kc2,
#                                   const cv::Mat & Rt, const cv::Mat & T, const cv::Point2d & p1, const cv::Point2d & p2,
#                                   cv::Point3d & p3d, double * distance)
def triangulate_stereo(K1, kc1, K2, kc2, Rt, T, p1, p2):
    # inp1 = np.zeros((1,1,2))#這邊應該是p1的點啦
    # inp2 = np.zeros((1,1,2))#這邊應該是p2的點吧
    # outp1 = cv2.undistortPoints(inp1, K1, kc1)
    # outp2 = cv2.undistortPoints(inp2, K2, kc2)
    # assert(outp1.type()==CV_64FC2 && outp1.rows==1 && outp1.cols==1)終止程式
    # u1 = np.array([inp1[0],inp1[1],1.0])
    # u2 = np.array([inp2[0],inp2[1],1.0])
    outp1 = cv2.undistortPoints(p1, K1, kc1) #根據相機參數和觀測到點座標位置計算實際座標位置
    outp2 = cv2.undistortPoints(p2, K2, kc2)
    u1 = np.array([outp1[0,0,0], outp1[0,0,1], 1.0])
    u2 = np.array([outp2[0,0,0], outp2[0,0,1], 1.0])
    # to world coordinates
    w1 = u1
    w2 = np.dot(Rt,(u2-T))
    # w2 = cv::Point3d(cv::Mat(Rt*(cv::Mat(u2) - T)));

    # world rays
    v1 = w1
    v2 = np.dot(Rt, u2)
    # compute ray-ray approximate intersection
    # p3d = np.zeros((3))
    p3d, distance = approximate_ray_intersection(v1, w1, v2, w2)
    return p3d, distance

In [6]:
def reconstruct(calib, pattern_list, return_p): # max dist 感覺是判斷
    # retrun_p = np.empty(shape=[3,3])
    for i in range(pattern_list.shape[0]):  # 每一組對應的proj-cam
        p2 = np.array([pattern_list[i][0], pattern_list[i][1]])  # proj_point
        p1 = np.array([pattern_list[i][2], pattern_list[i][3]])  # cam_point
        R_t = calib.R.T # transpose
        p, distance = triangulate_stereo(calib.cam_k, calib.cam_kc, calib.proj_k, calib.proj_kc, R_t, calib.T, p1, p2)
        return_p = np.append(return_p, [p], axis=0)
    return return_p

In [14]:
pointcloud = Pointcloud()

cam_k = np.array([[2.79785046e+03, 0.00000000e+00, 6.43670471e+02],
  [0, 2.84502856e+03, 1.07247008e+03],
  [0, 0, 1]])
cam_kc = np.array([[-0.89151335,  3.98951334, -0.07131889,  0.03804235, -8.48680502]])
proj_k = np.array([[1.96033608e+03, 0, 4.38923940e+02],
  [0, 5.98626108e+03, 5.77459005e+02],
  [0, 0, 1]])
proj_kc = np.array([[-9.43865235e+00,  1.02462776e+03, -8.85666070e-01, -5.39344800e-03, -1.89992224e+04]])
R = np.array([[0.99249327,  0.06323686, -0.10468148],
  [-0.10828452,  0.85223621, -0.511828],
  [ 0.05684695,  0.51932123,  0.85268628]])
T = np.array([-0.31667535, 36.92459112, 62.01244307])
cam_err = 0.166797
proj_err = 0.101493
stereo_err = 0.158868


calib = CalibrationData(cam_k, cam_kc, proj_k, proj_kc, R, T,
                 cam_err, proj_err, stereo_err)

# proj_size = np.array([1024, 768])
proj_size = np.array([1280, 800])
threshold = 25 #in scan3d-capture.exe


In [ ]:
folder = '/content/drive/MyDrive/lab/experiment/1225/'
text = 'output_m1.csv'


pattern_list = np.empty(shape=[0, 4])
with open(folder+text, newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        pattern_list = np.append(pattern_list, [[float(row[0]), float(row[1]), float(row[2]), float(row[3])]], axis=0)

pattern_list

In [10]:
def read_pro_txtfile(file_name):
  pattern_list = np.empty(shape=[0, 4])
  with open(file_name, newline='') as pro_file:
      for row in pro_file.readlines():
        p1, p2, p3, p4 = row.split(" ")
        pattern_list = np.append(pattern_list, [[float(p1), float(p2), float(p3), float(p4)]], axis=0)

  print(f'find {len(pattern_list)} points')
  return pattern_list

In [ ]:
return_p = np.empty(shape=[0, 3])
return_p = reconstruct(calib, pattern_list, return_p)

In [12]:
# 3D reconstruction
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(return_p)
o3d.io.write_point_cloud(folder+"1225_b_plane.ply", pcd)

True

In [18]:
folder = '0720_2'

for i in range(0, 1):
  ftext = f'{folder}/test_{i}_cut/project.txt'
  pattern_list = read_pro_txtfile(ftext)

  return_p = np.empty(shape=[0, 3])
  return_p = reconstruct(calib, pattern_list, return_p)
  print(f'construct {len(return_p)} 3D point')
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(return_p)
  o3d.io.write_point_cloud(f"{folder}/3D_{i}.ply", pcd)

find 28 points
construct 28 3D point
